In [1]:
!which python

/u/pw7nc/anaconda3/bin/python


In [1]:
import nltk
import spacy
import re
import json
import pandas as pd
import os
import numpy as np
from googletrans import Translator
from nltk.tokenize import sent_tokenize, word_tokenize, wordpunct_tokenize

# Read Data

In [2]:
# Load the whole filtered data
dir_path = '../Dataset/wine/'
# Load whole dataset (cleaned and filtered)
whole_review = []
cnt = 0
file_path = os.path.join(dir_path, 'whole_filtered.json')
with open(file_path) as f:
    for line in f:
        line_data = json.loads(line)
        user_id = line_data['user']
        item_id = line_data['item']
        rating = line_data['rating']
        review = line_data['review']
        whole_review.append([item_id, user_id, rating, review])
        cnt += 1
        if cnt % 100000 == 0:
            print("{} lines loaded.".format(cnt))
print("Finish loading whole cleaned filtered dataset, totally {} lines.".format(cnt))

100000 lines loaded.
200000 lines loaded.
300000 lines loaded.
Finish loading whole cleaned filtered dataset, totally 307746 lines.


In [12]:
# detector = Translator()
# cnt_none_english = 0
# none_english_idx = list()
# none_english_lans = list()

# for idx, rvw_data in enumerate(whole_review):
#     rvw_text = rvw_data[-1]
#     dec_lan = detector.detect(rvw_text)
#     if dec_lan.lang == "en":
#         pass
#     else:
#         cnt_none_english += 1
#         none_english_idx.append(idx)
#         none_english_lans.append(dec_lan.lang)
#     if (idx+1) % 10000 == 0:
#         print("{} lines".format(idx+1))


AttributeError: 'Translator' object has no attribute 'raise_Exception'

# Split the Data into Train/Test

In [3]:
user_group = dict()
for rvw_data in whole_review:
    user_id = rvw_data[1]
    if user_id not in user_group:
        user_group[user_id] = list()
    user_group[user_id].append(rvw_data)

In [4]:
len(user_group)

6080

In [6]:
user_group = {u: v for u, v in user_group.items()}

In [7]:
TRAIN_FILE = os.path.join(dir_path, 'train_filtered.json')
VALID_FILE = os.path.join(dir_path, 'valid_filtered.json')
TEST_FILE = os.path.join(dir_path, 'test_filtered.json')

In [19]:
import random
import math
from collections import Counter

VALID_RATIO = 0.00
TEST_RATIO = 0.20

train_lines, valid_lines, test_lines = [], [], []

train_item_counter = Counter()
valid_item_counter = Counter()
test_item_counter = Counter()

cnt_user_num = 0

for uid, lines in user_group.items():
    random.shuffle(lines)

    leng = len(lines)
    valid_num = math.floor(leng * VALID_RATIO)
    test_num = math.floor(leng * TEST_RATIO)

    valid_lines += lines[:valid_num]
    test_lines += lines[-test_num:]
    train_lines += lines[valid_num:-test_num]

    for rvw_data in lines[:valid_num]:
        iid = rvw_data[0]
        valid_item_counter.update([iid])
    for rvw_data in lines[-test_num:]:
        iid = rvw_data[0]
        test_item_counter.update([iid])
    for rvw_data in lines[valid_num:-test_num]:
        iid = rvw_data[0]
        train_item_counter.update([iid])

    cnt_user_num += 1
    if cnt_user_num % 1000 == 0:
        print("{} users".format(cnt_user_num))

1000 users
2000 users
3000 users
4000 users
5000 users
6000 users


In [20]:
print("Number of review instance in train: {0}\nNumber of review instance in valid: {1}\nNumber of review instance in test: {2}".format(len(train_lines), len(valid_lines), len(test_lines)))

Number of review instance in train: 248452
Number of review instance in valid: 0
Number of review instance in test: 59294


In [21]:
train_item_set = set(train_item_counter.keys())
valid_item_set = set(valid_item_counter.keys())
test_item_set = set(test_item_counter.keys())

In [23]:
for iid in test_item_set:
    if iid not in train_item_set:
        print(iid)
    else:
        if train_item_counter[iid] <= test_item_counter[iid]:
            print("iid: {}\tnum on train: {}\tnum on test: {}".format(
                iid, train_item_counter[iid], test_item_counter[iid]
            ))

iid: 224285	num on train: 4	num on test: 6
iid: 230214	num on train: 5	num on test: 6
iid: 239273	num on train: 6	num on test: 6
iid: 939626	num on train: 5	num on test: 5
iid: 872107	num on train: 5	num on test: 6
iid: 114304	num on train: 4	num on test: 6
iid: 958566	num on train: 5	num on test: 5
iid: 40294	num on train: 5	num on test: 5
iid: 927388	num on train: 8	num on test: 8
iid: 21771	num on train: 7	num on test: 8
iid: 105531	num on train: 8	num on test: 8
iid: 93074	num on train: 5	num on test: 5
iid: 279789	num on train: 5	num on test: 5
iid: 695611	num on train: 5	num on test: 5
iid: 1145618	num on train: 7	num on test: 7
iid: 523499	num on train: 5	num on test: 5
iid: 774881	num on train: 6	num on test: 6
iid: 670612	num on train: 5	num on test: 5
iid: 12379	num on train: 6	num on test: 6
iid: 528484	num on train: 8	num on test: 9
iid: 462952	num on train: 5	num on test: 5
iid: 691653	num on train: 7	num on test: 7
iid: 403389	num on train: 4	num on test: 7
iid: 443792	nu

In [ ]:
with open(TRAIN_FILE, 'w') as train_file, open(VALID_FILE, 'w') as valid_file, open(TEST_FILE, 'w') as test_file:
    for rvw_data in train_lines:
        row_iid = rvw_data[0]
        row_uid = rvw_data[1]
        row_rating = rvw_data[2]
        row_review = rvw_data[3]
        row_dict = {
            'user': row_uid,
            'item': row_iid,
            'rating': row_rating,
            'review': row_review
        }
        # dump this into file
        json.dump(row_dict, train_file)
        train_file.write("\n")
    for rvw_data in test_lines:
        row_iid = rvw_data[0]
        row_uid = rvw_data[1]
        row_rating = rvw_data[2]
        row_review = rvw_data[3]
        row_dict = {
            'user': row_uid,
            'item': row_iid,
            'rating': row_rating,
            'review': row_review
        }
        # dump this into file
        json.dump(row_dict, test_file)
        test_file.write("\n")

In [24]:
train_lines[0]

['1015705',
 '131074',
 93,
 'this wine is simply outstanding . the nose has concentrated , ripe black cherry , some dark fruits and beautiful french oak aromas of espresso and cocoa . some perfume and violets . a pleasing spice . already beginning to show a little integration and stewy complexity even at 3 years . the palate is perfectly balanced , with spice and a little jam ( not overdone ) on the backside . medium , velvety tannins , medium plus alcohol , good acidity . very balanced . long length . what an outstanding value . purchased for about $ 25 but this wine drinks like it is double that price . needless to say , outstanding qpr . its food pairing versatility alone makes it worth keeping a few bottles in the cellar . paired perfectly today with a simple mothers day barbecue , but you can match it with haute cuisine as well . outstanding .']